In [308]:
#from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os
import pandas as pd
import numpy as np

In [309]:
#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [310]:
#The df
untouched_csv = pd.read_csv('CSV/RAW_data.csv', sep=',')

In [311]:
#replace the direction column with the corresponding degree values
direction_map = {'N': 0,'NNE': 22.5,'NE': 45,'ENE': 67.5,'E': 90,'ESE': 112.5,'SE': 135,'SSE': 157.5,
            'S': 180,'SSW': 202.5,'SW': 225,'WSW': 247.5,'W': 270,'WNW': 292.5,'NW': 315,'NNW': 337.5
        }

In [312]:
untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map).infer_objects(copy = False)
#untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].astype(int)

C:\Users\Krupam Patel\AppData\Local\Temp\ipykernel_6672\3364744739.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map).infer_objects(copy = False)


In [313]:
#convert to Celsius
untouched_csv['Mainland Air Temp'] = round((untouched_csv['Mainland Air Temp']-32) * 5.0/9.0, 1)
untouched_csv['LBI Air Temp'] = round((untouched_csv['LBI Air Temp']-32) * 5.0/9.0, 1)


In [314]:
#rounds all colomns
untouched_csv['Humidity (%)'] = round(untouched_csv['Humidity (%)'], 1)
untouched_csv['Wind Speed (A)'] = round(untouched_csv['Wind Speed (A)'], 1)
untouched_csv['Gusting'] = round(untouched_csv['Gusting'], 1)
untouched_csv['Atmospheric Pressure (IN)'] = round(untouched_csv['Atmospheric Pressure (IN)'], 2)
untouched_csv['Precipitation Rate'] = round(untouched_csv['Precipitation Rate'], 2)
untouched_csv['Bay Temp'] = round(untouched_csv['Bay Temp'], 2)
untouched_csv['Salinity'] = round(untouched_csv['Salinity'], 2)
untouched_csv['LBI Air Temp'] = round(untouched_csv['LBI Air Temp'], 1)
untouched_csv['Ocean Temp'] = round(untouched_csv['Ocean Temp'], 1)

In [315]:
#determines if its a onshore breeze and adds a new column for it
onshore_degrees = [180, 135, 157.5, 90, 67.5, 45, 22.5]
untouched_csv['Onshore'] = untouched_csv['Direction (A)'].isin(onshore_degrees)
untouched_csv['Onshore'] = untouched_csv['Onshore'].astype(int)



In [316]:
#thresholds
ocean_thresh = 1.0
#bay_thresh = 1.67
wind_thresh = 7.0

In [317]:
#thresholds
#ocean_thresh = 1.0
#bay_thresh = 1.67
#wind_thresh = 7.0

#untouched_csv['rolling avg'] = untouched_csv['Bay Temp'].rolling(window=24).mean()

#Boolean math to check if it is upwelling or not - OLD
#untouched_csv['bay_bool'] = (abs(untouched_csv['Bay Temp'] - untouched_csv['rolling avg']) > bay_thresh).astype(int)
#untouched_csv['ocean_bool'] = (abs(untouched_csv["Bay Temp"] - untouched_csv["Ocean Temp"]) > ocean_thresh).astype(int)
#untouched_csv['big_wind'] = (untouched_csv['Wind Speed (A)'] > wind_thresh).astype(int)
#untouched_csv["upwelling_flag"] = ((untouched_csv['bay_bool'] == 1) & (untouched_csv['ocean_bool'] == 1) & (untouched_csv['big_wind'] == 1)).astype(int)
"""Explain this here"""

#Drop the temporary colomns
#untouched_csv.drop(['bay_bool', 'ocean_bool', 'big_wind', 'rolling avg'], axis=1, inplace=True)
	
	
    

'Explain this here'

In [318]:
#average of two lowest ocean points in a day
untouched_csv['ocean_min1'] = untouched_csv['Ocean Temp'].rolling(24).min()
untouched_csv['ocean_min2'] = (
    untouched_csv['Ocean Temp'].rolling(24).apply(lambda x: np.sort(x)[1], raw=False)
)
untouched_csv['ocean_min'] = (untouched_csv['ocean_min1']+untouched_csv['ocean_min2'])/2

#Check if its a upwelling direction
upwell_wind = [135, 165, 180, 270]
untouched_csv['upwell_wind'] = untouched_csv['Direction (A)'].isin(upwell_wind)
untouched_csv['upwell_wind'] = untouched_csv['upwell_wind'].astype(int)

#bools:
#   There are two tide cycles in a day, if you take the two lowest values and average them together. 
#   Taking the absolute value of the difference of the low values and the current temperature and if this is bigger than the threshold the Ocean boolean is true. 
untouched_csv['big_wind'] = (untouched_csv['Wind Speed (A)'] > wind_thresh).astype(int)
untouched_csv['upwell_wind'] = (untouched_csv['upwell_wind'] == 1).astype(int)
untouched_csv['ocean_bool'] = (abs(untouched_csv['ocean_min'] - untouched_csv['Ocean Temp']) > ocean_thresh).astype(int)
untouched_csv["upwelling_flag"] = ((untouched_csv['ocean_bool'] == 1) & (untouched_csv['big_wind'] == 1)).astype(int)

#drop the colomns
untouched_csv.drop(['ocean_bool', 'big_wind', 'ocean_min1', 'ocean_min2', 'upwell_wind', 'ocean_min'], axis=1, inplace=True)


In [319]:
#Convert data into Date time and create date filter
untouched_csv['Date'] = pd.to_datetime(untouched_csv['Date'])

"""
Use this in future if data set needs specific dates
prediction = data.loc{
    (untouched_csv['Date'] > datetime(x, x, x)) &
    (untouched_csv['Date'] < datetime(x, x, x,))
}
"""

"\nUse this in future if data set needs specific dates\nprediction = data.loc{\n    (untouched_csv['Date'] > datetime(x, x, x)) &\n    (untouched_csv['Date'] < datetime(x, x, x,))\n}\n"

In [320]:
#Save this data frame to a Finished csv

In [321]:


#info on csv
#print(untouched_csv.info())

#Number of upwelling events
#num_upwelling_events = untouched_csv['upwelling_flag'].sum()
#print("Number of upwelling hours:", num_upwelling_events)


#Find which rows where upwelling
#upwelling_rows = untouched_csv[untouched_csv['upwelling_flag'] == 1]
#print(upwelling_rows)


print(untouched_csv.to_string())
untouched_csv.head()

           Date  Hr  Mainland Air Temp  Humidity (%)  Direction (A)  Wind Speed (A)  Gusting  Atmospheric Pressure (IN)  Precipitation Rate  Bay Temp  Salinity  LBI Air Temp  Ocean Temp  Onshore  upwelling_flag
0    2025-06-01   0               11.8          88.0          270.0             6.8     12.1                      29.54                0.00     17.52     28.34          13.4        14.5        0               0
1    2025-06-01   1               11.2          86.0          270.0             5.5     12.5                      29.55                0.00     17.43     27.89          13.0        14.5        0               0
2    2025-06-01   2               10.4          83.0          247.5             4.5      9.3                      29.57                0.00     17.42     27.90          12.8        14.4        0               0
3    2025-06-01   3               10.1          79.0          225.0             2.6      4.0                      29.59                0.00     17.34     28

,Date,Hr,Mainland Air Temp,Humidity (%),Direction (A),Wind Speed (A),Gusting,Atmospheric Pressure (IN),Precipitation Rate,Bay Temp,Salinity,LBI Air Temp,Ocean Temp,Onshore,upwelling_flag
0,2025-06-01,0,11.8,88.0,270.0,6.8,12.1,29.54,0.0,17.52,28.34,13.4,14.5,0,0
1,2025-06-01,1,11.2,86.0,270.0,5.5,12.5,29.55,0.0,17.43,27.89,13.0,14.5,0,0
2,2025-06-01,2,10.4,83.0,247.5,4.5,9.3,29.57,0.0,17.42,27.90,12.8,14.4,0,0
3,2025-06-01,3,10.1,79.0,225.0,2.6,4.0,29.59,0.0,17.34,28.57,12.2,14.4,0,0
4,2025-06-01,4,8.7,86.0,247.5,3.9,6.2,29.60,0.0,17.23,28.20,12.2,14.4,0,0


In [322]:
#Prepare for the LSTM model (Sequential)
